# QKD BB84 Protocol Implementation
Arjun Bhamra

### Imports

In [5]:
# make sure to activate your qbraid environment! 
from qiskit import *
from qiskit import Aer
import quantuminspire
from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI

### Authentication for Quantum Inspire

In [3]:
from quantuminspire.credentials import save_account
from quantuminspire.qiskit import QI
save_account('411ae2cc3197826c315d30bb9315522048307de0')
from quantuminspire.credentials import get_token_authentication
auth = get_token_authentication()
QI.set_authentication()

qi = QuantumInspireAPI(r'https://api.quantum-inspire.com', auth, "QKD-BB84-Implementation")

### BB84 Protocol Code

#### Helper Functions and Supplemental Imports

In [4]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np

In [5]:
def encode(bits, bases): #function to encode the results of user 1's bits and bases
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        else: # Prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

In [6]:
def decode(message, bases):
    
    #change to QUANTUM INSPIRE BACKEND LATER
    #backend = Aer.get_backend('aer_simulator')
    simulator = QI.get_backend('QX single-node simulator')

    measurements = []
    
    for i in range(n): #iterating through all basis/message circuit pairs
        qc = QuantumCircuit(1, 1)
        if bases[i] == 0: #Z basis measurement
            message[i].measure(0, 0)
        if bases[i] == 1: #X basis measurement
            message[i].h(0)
            message[i].measure(0, 0)
            
        qi_job = execute(message[i], backend = simulator, shots = 1)
        qi_result = qi_job.result()

        measured_bit = int(qi_result.get_memory()[0])
        measurements.append(measured_bit)
        
        
    
    return measurements

In [7]:
def common_bases(u1_bases, u2_bases, u1_bits):
    good_bits = []
    for i in range(n):
        if u1_bases[i]==u2_bases[i]:
            good_bits.append(u1_bits[i])
            
    return good_bits

In [8]:
def sample_bits(bits, selection):
    sample = []
    for i in selection:
        # use np.mod to make sure the
        # bit we sample is always in 
        # the list range
        i = np.mod(i, len(bits))
        # pop(i) removes the element of the
        # list at index 'i'
        sample.append(bits.pop(i))
    return sample


#alternative is to use np.random.choice for random selection of list

#### BB84 Runner

In [11]:
#Using the Qiskit Code (from the Documentation)
np.random.seed(seed=0)

n=100 #100 qubit message from Alice

#Step 1: User 1 Generates Bits
u1_bits = randint(2, size=n)

#Step 2: User 1 Generates Bases (0 = Z basis, 1 = X basis)
u1_bases = randint(2, size=n)
message = encode(u1_bits, u1_bases) #a list of circuits with the correct basis applied

#Step 3: User 2 Generates Bases (0 = Z basis, 1 = X basis) and measures User 1's message
u2_bases = randint(2, size=n)
u2_results = decode(message, u2_bases) 

#Step 4: Remove the bases that aren't common
u1_key = common_bases(u1_bases, u2_bases, u1_bits)
u2_key = common_bases(u1_bases, u2_bases, u2_results)

#Step 5: Sample a random selection of the bits and compare values to results to verify the security of 
#the information exchange
sample_size = 10
bit_selection = randint(n, size=sample_size)
u1_sample = sample_bits(u1_key, bit_selection)
u2_sample = sample_bits(u2_key, bit_selection)

In [12]:
if u1_sample == u2_sample:
    print("You did it!")

You did it!


In [73]:
print(u1_key)
print(u2_key)
print("key length = %i" % len(u1_key))

[0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0]
[0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0]
key length = 38


### Huffman Encoding Algorithm

In [146]:
    # Node of a Huffman Tree  
    class Nodes:  
        def __init__(self, probability, symbol, left = None, right = None):  
            # probability of the symbol  
            self.probability = probability  
      
            # the symbol  
            self.symbol = symbol  
      
            # the left node  
            self.left = left  
      
            # the right node  
            self.right = right  
      
            # the tree direction (0 or 1)  
            self.code = ''  
      
    """ A supporting function in order to calculate the probabilities of symbols in specified data """  
    def CalculateProbability(the_data):  
        the_symbols = dict()  
        for item in the_data:  
            if the_symbols.get(item) == None:  
                the_symbols[item] = 1  
            else:   
                the_symbols[item] += 1       
        return the_symbols  
      
    """ A supporting function in order to print the codes of symbols by travelling a Huffman Tree """  
    the_codes = dict()  
      
    def CalculateCodes(node, value = ''):  
        # a huffman code for current node  
        newValue = value + str(node.code)  
      
        if(node.left):  
            CalculateCodes(node.left, newValue)  
        if(node.right):  
            CalculateCodes(node.right, newValue)  
      
        if(not node.left and not node.right):  
            the_codes[node.symbol] = newValue  
               
        return the_codes  
      
    """ A supporting function in order to get the encoded result """  
    def OutputEncoded(the_data, coding):  
        encodingOutput = []  
        for element in the_data:  
            # print(coding[element], end = '')  
            encodingOutput.append(coding[element])  
              
        the_string = ''.join([str(item) for item in encodingOutput])      
        return the_string  
              
    """ A supporting function in order to calculate the space difference between compressed and non compressed data"""      
    def TotalGain(the_data, coding):  
        # total bit space to store the data before compression  
        beforeCompression = len(the_data) * 8  
        afterCompression = 0  
        the_symbols = coding.keys()  
        for symbol in the_symbols:  
            the_count = the_data.count(symbol)  
            # calculating how many bit is required for that symbol in total  
            afterCompression += the_count * len(coding[symbol])  
        print("Space usage before compression (in bits):", beforeCompression)  
        print("Space usage after compression (in bits):",  afterCompression)  
      
    def HuffmanEncoding(the_data):  
        symbolWithProbs = CalculateProbability(the_data)  
        the_symbols = symbolWithProbs.keys()  
        the_probabilities = symbolWithProbs.values()  
        print("symbols: ", the_symbols)  
        print("probabilities: ", the_probabilities)  
          
        the_nodes = []  
          
        # converting symbols and probabilities into huffman tree nodes  
        for symbol in the_symbols:  
            the_nodes.append(Nodes(symbolWithProbs.get(symbol), symbol))  
          
        while len(the_nodes) > 1:  
            # sorting all the nodes in ascending order based on their probability  
            the_nodes = sorted(the_nodes, key = lambda x: x.probability)  
            # for node in nodes:    
            #      print(node.symbol, node.prob)  
          
            # picking two smallest nodes  
            right = the_nodes[0]  
            left = the_nodes[1]  
          
            left.code = 0  
            right.code = 1  
          
            # combining the 2 smallest nodes to create new node  
            newNode = Nodes(left.probability + right.probability, left.symbol + right.symbol, left, right)  
          
            the_nodes.remove(left)  
            the_nodes.remove(right)  
            the_nodes.append(newNode)  
                  
        huffmanEncoding = CalculateCodes(the_nodes[0])  
        print("symbols with codes", huffmanEncoding)  
        TotalGain(the_data, huffmanEncoding)  
        encodedOutput = OutputEncoded(the_data,huffmanEncoding)  
        return encodedOutput, the_nodes[0]  
       
    def HuffmanDecoding(encodedData, huffmanTree):  
        treeHead = huffmanTree  
        decodedOutput = []  
        for x in encodedData:  
            if x == '1':  
                huffmanTree = huffmanTree.right     
            elif x == '0':  
                huffmanTree = huffmanTree.left  
            try:  
                if huffmanTree.left.symbol == None and huffmanTree.right.symbol == None:  
                    pass  
            except AttributeError:  
                decodedOutput.append(huffmanTree.symbol)  
                huffmanTree = treeHead  
              
        string = ''.join([str(item) for item in decodedOutput])  
        return string  
    
# the_data = input("What data would you like to encode?")
# print(the_data)  
# encoding, the_tree = HuffmanEncoding(the_data)  
# print("Encoded output", encoding)  
# print("Decoded Output", HuffmanDecoding(encoding, the_tree)) 
 

### BB84 With Huffman Encoding

#### Encoding with Huffman Algorithm

In [151]:
the_data = input("What data/input would you like to encode and send?")
encoding, the_tree = HuffmanEncoding(the_data)
print("Encoded value: " + encoding)
#print(list(encoding))
encoded_list = []
for i in list(encoding):
    encoded_list.append(int(i))
print(encoded_list)

What data/input would you like to encode and send? hello!


symbols:  dict_keys(['h', 'e', 'l', 'o', '!'])
probabilities:  dict_values([1, 1, 2, 1, 1])
symbols with codes {'e': '000', 'h': '001', 'l': '01', '!': '10', 'o': '11'}
Space usage before compression (in bits): 48
Space usage after compression (in bits): 14
Encoded value: 00100001011110
[0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0]


#### BB84 and Verifying Channel Security with Keys

In [152]:
#Step 1: User 1 Generates Bits
#u1_bits = randint(2, size=n)
n = len(encoded_list)
u1_bits = encoded_list

#Step 2: User 1 Generates Bases (0 = Z basis, 1 = X basis)
u1_bases = randint(2, size=n)
message = encode(u1_bits, u1_bases) #a list of circuits with the correct basis applied

#Step 3: User 2 Generates Bases (0 = Z basis, 1 = X basis) and measures User 1's message
u2_bases = randint(2, size=n)
u2_results = decode(message, u2_bases) 

#Step 4: Remove the bases that aren't common
u1_key = common_bases(u1_bases, u2_bases, u1_bits)
u2_key = common_bases(u1_bases, u2_bases, u2_results)

#Step 5: Sample a random selection of the bits and compare values to results to verify the security of 
#the information exchange
sample_size = 5
bit_selection = randint(n, size=sample_size)
u1_sample = sample_bits(u1_key, bit_selection)
u2_sample = sample_bits(u2_key, bit_selection)

In [153]:
if u1_sample == u2_sample:
    print("You did it!")
print(u1_key)
print(u2_key)
print("key length = %i" % len(u1_key))

You did it!
[1, 1]
[1, 1]
key length = 2


#### Huffman Decoding

In [150]:
keystring = ""
for num in encoded_list:
    keystring+=str(num)
    
print("Decoded Output:", HuffmanDecoding(keystring, the_tree))   

Decoded Output: hello!
